In [1]:
from datetime import datetime
import pandas as pd
from selenium import webdriver


week_str = '20221106'
# datetime object containing current date and time
now = datetime.now()


d_id_to_city = {'1078741': 'Chicago, IL', 
                'RunDMRC': 'Dallas, TX', 
                'lamrc': 'Los Angeles, CA',
                'nycmrc': 'New York, NY',
                '1020331': 'San Francisco, CA',
                'detroitmrc': 'Detroit, MI',
                'dcmuslimsrun': 'Washington, DC',
                '1088734': 'Houston, TX',
                'seattlemrc': 'Seattle, WA',
                'sdmrc': 'San Diego, CA',
                }

d_club_data = {}
for club in d_id_to_city:
    driver = webdriver.Chrome()
    location = d_id_to_city[club]
    driver.get('https://www.strava.com/clubs/' + club + '/leaderboard')
    
    dfs = pd.read_html(driver.page_source)
    
    d_club_data[location] = dfs[-1]
    print(location, len(dfs[-1]))
    d_club_data[location]['timestamp'] = now
    d_club_data[location]['location'] = location
    
    driver.close()


Chicago, IL 16
Dallas, TX 46
Los Angeles, CA 16
New York, NY 39
San Francisco, CA 34
Detroit, MI 8
Washington, DC 6
Houston, TX 1
Seattle, WA 3
San Diego, CA 2


In [2]:
from math import floor

# handle LA

def fixtime(t):
    if 'h' in t[0]:
        return (float(t[0][:-1]) * 60) + float(t[1][:-1])
    return float(t[0][:-1])

d_club_data['Los Angeles, CA']['time_hold'] = d_club_data['Los Angeles, CA']['Time'].map(lambda x: x.split(" "))
d_club_data['Los Angeles, CA']['time_hold'] = d_club_data['Los Angeles, CA']['time_hold'].map(lambda x: fixtime(x))
d_club_data['Los Angeles, CA']['dist_LA'] = d_club_data['Los Angeles, CA']['Distance'].map(lambda x: float(x[:-3]))
d_club_data['Los Angeles, CA']['Avg. Pace'] = d_club_data['Los Angeles, CA']['time_hold'] / d_club_data['Los Angeles, CA']['dist_LA']
d_club_data['Los Angeles, CA']['Avg. Pace'] = d_club_data['Los Angeles, CA']['Avg. Pace'].map(lambda x: str(floor(x)) + ":" + str(int((x - floor(x)) * 60)) + " /km")
d_club_data['Los Angeles, CA']

,Rank,Athlete,Distance,Activities,Elev. Gain,Time,timestamp,location,time_hold,dist_LA,Avg. Pace
0,1,Sanaa O.,26.7 km,4,245 m,2h 38m,2022-11-05 14:47:35.035550,"Los Angeles, CA",158.0,26.7,5:55 /km
1,2,Ahmed A.,14.5 km,3,163 m,2h 52m,2022-11-05 14:47:35.035550,"Los Angeles, CA",172.0,14.5,11:51 /km
2,3,shajee’ah 🐿.,13.1 km,2,93 m,1h 13m,2022-11-05 14:47:35.035550,"Los Angeles, CA",73.0,13.1,5:34 /km
3,4,Ayesha H.,12.7 km,2,122 m,1h 31m,2022-11-05 14:47:35.035550,"Los Angeles, CA",91.0,12.7,7:9 /km
4,5,Zohair A.,10.0 km,1,3 m,1h 3m,2022-11-05 14:47:35.035550,"Los Angeles, CA",63.0,10.0,6:17 /km
5,6,Didi R.,10.0 km,1,29 m,59m,2022-11-05 14:47:35.035550,"Los Angeles, CA",59.0,10.0,5:54 /km
6,7,Ayan J.,8.2 km,2,--,1h 3m,2022-11-05 14:47:35.035550,"Los Angeles, CA",63.0,8.2,7:40 /km
7,8,Sohaib N.,6.6 km,1,111 m,36m,2022-11-05 14:47:35.035550,"Los Angeles, CA",36.0,6.6,5:27 /km
8,9,Nida A.,5.4 km,1,39 m,36m,2022-11-05 14:47:35.035550,"Los Angeles, CA",36.0,5.4,6:39 /km
9,10,𝘈𝘢𝘮𝘪𝘯𝘢𝘩 𝘒.,5.3 km,2,28 m,36m,2022-11-05 14:47:35.035550,"Los Angeles, CA",36.0,5.3,6:47 /km


In [3]:
df_all = pd.concat([d_club_data[k] for k in d_club_data])
df_all = df_all[df_all['Distance'] != 'There are no results.']
df_all['dist_float'] = df_all['Distance'].map(lambda x: float(x[:-3]))
df_all['avg_pace_float'] = df_all['Avg. Pace'].map(lambda x: x[:-4].split(":"))
df_all['avg_pace_km_min'] = df_all['avg_pace_float'].map(lambda x: float((60 * int(x[0])) + int(x[1])) / 60)
df_all['time_spent'] = df_all['avg_pace_km_min'] * df_all['dist_float']

df_all = df_all.sort_values(by=['dist_float'], ascending=False)
df_all['National Rank'] = [x for x in range(1, len(df_all) + 1)]
df_all['Location Rank'] = df_all['Rank']
df_all['elevation_gain'] = df_all['Elev. Gain']
df_all['elevation_gain'] = df_all['elevation_gain'].map(lambda x: float(x[:-2]) if x != '--' else 0)
df_all.to_csv("rawdata/" + "raw_data_" + week_str + ".csv", index=False)

df_fts = df_all[['Athlete', 'dist_float', 'avg_pace_km_min', 'elevation_gain', 'time_spent', 'location']].copy(deep=True)

# dist rank
df_fts = df_fts.sort_values(by=['dist_float'], ascending=False)
df_fts['dist_rank'] = [x for x in range(1, len(df_fts) + 1)]

# avg_pace_km_min rank
df_fts = df_fts.sort_values(by=['avg_pace_km_min'], ascending=True)
df_fts['pace_rank'] = [x for x in range(1, len(df_fts) + 1)]

# elevation_gain rank
df_fts = df_fts.sort_values(by=['elevation_gain'], ascending=False)
df_fts['elevation_rank'] = [x for x in range(1, len(df_fts) + 1)]

df_all = df_all[['National Rank', 'Athlete', 'Distance', 'Runs', 'Longest', 'elevation_gain', 'timestamp', 'location', 'dist_float', 'Location Rank']]

In [4]:
df_all.sample(5)

,National Rank,Athlete,Distance,Runs,Longest,elevation_gain,timestamp,location,dist_float,Location Rank
41,148,Lauren K.,4.4 km,2.0,3.4 km,13.0,2022-11-05 14:47:35.035550,"Dallas, TX",4.4,42
38,141,Neelam Z.,4.9 km,1.0,4.9 km,8.0,2022-11-05 14:47:35.035550,"Dallas, TX",4.9,39
12,61,Mounir R.,12.6 km,2.0,6.9 km,26.0,2022-11-05 14:47:35.035550,"San Francisco, CA",12.6,13
9,62,Saad A.,12.5 km,4.0,3.4 km,80.0,2022-11-05 14:47:35.035550,"New York, NY",12.5,10
29,132,Tariq U.,5.0 km,1.0,5.0 km,27.0,2022-11-05 14:47:35.035550,"New York, NY",5.0,30


In [5]:
df_fts['National Rank'] = (df_fts['dist_rank'] + df_fts['pace_rank'] + df_fts['elevation_rank']) / 3
df_fts = df_fts.sort_values(by=['National Rank'], ascending=True)
df_fts['National Rank'] = [x for x in range(1, len(df_fts) + 1)]
df_fts = df_fts[['National Rank', 'Athlete', 'dist_float', 'avg_pace_km_min', 'elevation_gain', 'time_spent', 'location']]
df_fts

,National Rank,Athlete,dist_float,avg_pace_km_min,elevation_gain,time_spent,location
0,1,Shehryar K.,94.1,4.966667,277.0,467.363333,"New York, NY"
2,2,Yousef S.,37.8,4.916667,184.0,185.850000,"Chicago, IL"
1,3,Nofil S.,43.1,5.316667,209.0,229.148333,"Dallas, TX"
0,4,Nofil S.,43.1,5.316667,209.0,229.148333,"San Francisco, CA"
2,5,Zakariya T.,35.2,5.183333,172.0,182.453333,"Detroit, MI"
...,...,...,...,...,...,...,...
35,167,Reaa A.,3.2,8.466667,0.0,27.093333,"New York, NY"
45,168,Mohammed Mustafa S.,2.6,7.850000,0.0,20.410000,"Dallas, TX"
14,169,Deena K.,2.1,7.766667,0.0,16.310000,"Chicago, IL"
38,170,J P.,0.8,8.766667,0.0,7.013333,"New York, NY"


In [6]:
df_fts.groupby(['location']).sum('dist_float')

,National Rank,dist_float,avg_pace_km_min,elevation_gain,time_spent
location,,,,,
"Chicago, IL",1522,257.1,103.383333,384.0,1565.393333
"Dallas, TX",4010,661.6,315.900000,2716.0,4374.506667
"Detroit, MI",696,143.3,51.533333,440.0,840.981667
"Houston, TX",46,29.2,6.933333,36.0,202.453333
"Los Angeles, CA",1507,138.2,107.016667,920.0,941.840000
"New York, NY",3346,437.3,244.333333,1616.0,2612.915000
"San Diego, CA",272,13.0,16.400000,15.0,93.840000
"San Francisco, CA",2606,465.9,213.233333,2105.0,2839.625000
"Seattle, WA",234,39.0,17.716667,450.0,232.816667


In [7]:
df_fts.to_csv('MRC_week_' + week_str + '.csv', index=False)

In [8]:
df_all.groupby(['location']).sum('dist_float')

,National Rank,Runs,elevation_gain,dist_float,Location Rank
location,,,,,
"Chicago, IL",1328,46.0,384.0,257.1,136
"Dallas, TX",3478,109.0,2716.0,661.6,1081
"Detroit, MI",653,26.0,440.0,143.3,36
"Houston, TX",18,3.0,36.0,29.2,1
"Los Angeles, CA",1639,0.0,920.0,138.2,136
"New York, NY",3871,71.0,1616.0,437.3,780
"San Diego, CA",239,3.0,15.0,13.0,3
"San Francisco, CA",2738,73.0,2105.0,465.9,595
"Seattle, WA",261,6.0,450.0,39.0,6
